In [2]:
# Install the required python packages
!pip install pandas
!pip install SQLAlchemy
!pip install "snowflake-connector-python[pandas]"
!pip install snowflake-sqlalchemy
!pip install gspread
!pip install gspread-dataframe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00


In [3]:
# Additional packages for geocode & google translate
!pip install geopy
!pip install dask
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.7 MB/s eta 0:00:00


In [17]:
from geopy.geocoders import Nominatim
from pprint import pprint

# Instantiate a new Nominatim client
app = Nominatim(user_agent="tutorial")

# Get location raw data from the user
your_loc = input("Enter your location: ")
location = app.geocode(your_loc).raw

# Print raw data
pprint(location)
print("\n\n")
pprint(app.reverse((52.23614,21.00817)).raw['address'])

Enter your location: 52.23614,21.00817
{'addresstype': 'amenity',
 'boundingbox': ['52.2357458', '52.2362462', '21.0081529', '21.0084925'],
 'class': 'amenity',
 'display_name': 'Marszałkowska, Centrum, Śródmieście Północne, Śródmieście, '
                 'Warszawa, województwo mazowieckie, 00-005, Polska',
 'importance': 8.366292820625647e-05,
 'lat': '52.236080799999996',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
            'http://osm.org/copyright',
 'lon': '21.00827373998952',
 'name': '',
 'osm_id': 1107718234,
 'osm_type': 'way',
 'place_id': 158262715,
 'place_rank': 30,
 'type': 'parking'}



{'ISO3166-2-lvl4': 'PL-14',
 'city': 'Warszawa',
 'country': 'Polska',
 'country_code': 'pl',
 'neighbourhood': 'Centrum',
 'postcode': '00-005',
 'quarter': 'Śródmieście Północne',
 'road': 'Marszałkowska',
 'state': 'województwo mazowieckie',
 'suburb': 'Śródmieście'}


In [26]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.point import Point
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import pd_writer
import time
import dask.dataframe as dd

start_time = time.time()

geolocator = Nominatim(user_agent="tutorial")

engine = create_engine(URL(
                    account = 'hrwnkma-yd18638',
                    user = 'ranjeet270',
                    password = 'Ranjeet@123',
                    database = 'demo',
                    schema = 'public',
                    warehouse = 'demo_wh'))

In [27]:
with engine.connect() as conn:
    try:
        query = """ SELECT RN, concat(latitude,',',longitude) as LOCATION
                    FROM (SELECT RN
                          , SUBSTR(location, REGEXP_INSTR(location,' ',1,4)+1) AS LATITUDE
                          , SUBSTR(location, REGEXP_INSTR(location,' ',1,1)+1, (REGEXP_INSTR(location,' ',1,2) - REGEXP_INSTR(location,' ',1,1) - 1) ) AS LONGITUDE
                        FROM otodom_data_flatten ORDER BY rn) """
        print("--- %s seconds ---" % (time.time() - start_time))

        df = pd.read_sql(query,conn)

        df.columns = map(lambda x: str(x).upper(), df.columns)

        ddf = dd.from_pandas(df,npartitions=100)
        print(ddf.head(5,npartitions=-1))

        ddf['ADDRESS'] = ddf['LOCATION'].apply(lambda x: geolocator.reverse(x).raw['address'],meta=(None, 'str'))
        print("--- %s seconds ---" % (time.time() - start_time))

        pandas_df = ddf.compute()
        print(pandas_df.head())
        print("--- %s seconds ---" % (time.time() - start_time))

        pandas_df.to_sql('otodom_data_flatten_address', con=engine, if_exists='append', index=False, chunksize=16000, method=pd_writer)
    except Exception as e:
        print('--- Error --- ',e)
    finally:
        conn.close()
engine.dispose()

print("--- %s seconds ---" % (time.time() - start_time))

--- 39.72012376785278 seconds ---
   RN                             LOCATION
0   1                    52.23614,21.00817
1   2                  52.336575,21.029306
2   3  51.10710682881388,16.94346882507325
3   4                    50.10361,20.00665
4   5                  52.336575,21.029306
--- 43.99381756782532 seconds ---


--- Error ---  HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=52.20997&lon=20.96727&format=json&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


--- 52.55512452125549 seconds ---


In [22]:
conn

In [25]:
 with engine.connect() as conn:
  try:
    query = """ SELECT RN, concat(latitude,',',longitude) as LOCATION
                    FROM (SELECT RN
                          , SUBSTR(location, REGEXP_INSTR(location,' ',1,4)+1) AS LATITUDE
                          , SUBSTR(location, REGEXP_INSTR(location,' ',1,1)+1, (REGEXP_INSTR(location,' ',1,2) - REGEXP_INSTR(location,' ',1,1) - 1) ) AS LONGITUDE
                        FROM otodom_data_flatten ORDER BY rn  ) """
    print("--- %s seconds ---" % (time.time() - start_time))

    df = pd.read_sql(query,conn)
    df.columns = map(lambda x: str(x).upper(), df.columns)

    print(df.head())


  except Exception as e:
        print('--- Error --- ',e)
  finally:
        conn.close()



--- 420.61012268066406 seconds ---
   RN                             LOCATION
0   1                    52.23614,21.00817
1   2                  52.336575,21.029306
2   3  51.10710682881388,16.94346882507325
3   4                    50.10361,20.00665
4   5                  52.336575,21.029306


In [28]:
df

,RN,LOCATION
0,1,"52.23614,21.00817"
1,2,"52.336575,21.029306"
2,3,"51.10710682881388,16.94346882507325"
3,4,"50.10361,20.00665"
4,5,"52.336575,21.029306"
...,...,...
62811,62812,"52.16044,21.0784"
62812,62813,"51.07641572477857,17.017576468725604"
62813,62814,"50.2559,19.0168"
62814,62815,"52.1570051,21.07894"


In [29]:
ddf['ADDRESS'] = ddf['LOCATION'].apply(lambda x: geolocator.reverse(x).raw['address'],meta=(None, 'str'))

In [31]:
pandas_df = ddf.loc[0].compute()
print(pandas_df.head())
print("--- %s seconds ---" % (time.time() - start_time))

   RN           LOCATION                                            ADDRESS
0   1  52.23614,21.00817  {'road': 'Marszałkowska', 'neighbourhood': 'Ce...
--- 1954.9464910030365 seconds ---


In [36]:
# pandas_df.to_sql('otodom_data_flatten_address', con=engine, if_exists='append', index=False, chunksize=16000, method=pd_writer)

In [35]:
# df['ADDRESS'] = df['LOCATION'].apply(lambda x: geolocator.reverse(x).raw['address'])

# Translate polish title to english

In [ ]:
import pandas as pd
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import pd_writer
import time
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

start_time = time.time()

engine = create_engine(URL(
                    account = 'ceiknnc-keb24333',
                    user = 'thoufiq',
                    password = 'Admin$1234',
                    database = 'demo',
                    schema = 'public',
                    warehouse = 'demo_wh'))

with engine.connect() as conn:
    try:
        query = """ SELECT RN, TITLE FROM otodom_data_flatten ORDER BY rn limit 300"""

        df = pd.read_sql(query,conn)

        gc = gspread.service_account()

        loop_counter = 0
        chunk_size = 100
        file_name = 'OTODOM_ANALYSIS_'
        user_email = 'demo.tfq@gmail.com'

        for i in range(0,len(df),chunk_size):
            loop_counter += 1
            df_in = df.iloc[i:(i+chunk_size), :]

            spreadsheet_title = file_name + str(loop_counter)
            try:
                locals()['sh'+str(loop_counter)] = gc.open(spreadsheet_title)
            except gspread.SpreadsheetNotFound:
                locals()['sh'+str(loop_counter)] = gc.create(spreadsheet_title)

            locals()['sh'+str(loop_counter)].share(user_email, perm_type='user', role='writer')
            wks = locals()['sh'+str(loop_counter)].get_worksheet(0)
            wks.resize(len(df_in)+1)
            set_with_dataframe(wks, df_in)

            column = 'C'   # Column to apply the formula
            start_row = 2  # Starting row to apply the formula
            end_row = wks.row_count   # Ending row to apply the formula
            cell_range = f'{column}{start_row}:{column}{end_row}'
            curr_row = start_row
            cell_list = wks.range(cell_range)

            for cell in cell_list:
                cell.value = f'=GOOGLETRANSLATE(B{curr_row},"pl","en")'
                curr_row += 1

            # Update the worksheet with the modified cells
            wks.update_cells(cell_list, value_input_option='USER_ENTERED')

            print(f'Spreadsheet {spreadsheet_title} created!')

            df_log = pd.DataFrame({'ID':[loop_counter], 'SPREADSHEET_NAME':[spreadsheet_title]})
            df_log.to_sql('otodom_data_log', con=engine, if_exists='append', index=False, chunksize=16000, method=pd_writer)


            # df_out = get_as_dataframe(wks, usecols=[0,1,2,3], nrows=end_row, header=None, skiprows=1)
            # print(f'Spreadsheet {locals()["sh"+str(loop_counter)]} loaded back to DataFrame!')

            # df_out.columns = ['RN', 'TITLE', 'LOCATION', 'TITLE_ENG']

            # df_out.to_sql('otodom_data_transformed', con=engine, if_exists='append', index=False, chunksize=16000, method=pd_writer)

    except Exception as e:
        print('--- Error --- ',e)
    finally:
        conn.close()
engine.dispose()

print("--- %s seconds ---" % (time.time() - start_time))

# Upload data into snowflake database



In [ ]:
import pandas as pd
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import pd_writer
import time
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

start_time = time.time()

engine = create_engine(URL(
                    account = 'ceiknnc-keb24333',
                    user = 'thoufiq',
                    password = 'Admin$1234',
                    database = 'demo',
                    schema = 'public',
                    warehouse = 'demo_wh'))

with engine.connect() as conn:
    try:
        query = """ SELECT ID, SPREADSHEET_NAME FROM otodom_data_log """
        df = pd.read_sql(query,conn)
        df.columns = map(lambda x: str(x).upper(), df.columns)

        gc = gspread.service_account()
        loop_counter = 0

        for index, row in df.iterrows():
            loop_counter += 1
            locals()['sh'+str(loop_counter)] = gc.open(row['SPREADSHEET_NAME'])
            wks = locals()['sh'+str(loop_counter)].get_worksheet(0)
            df_out = get_as_dataframe(wks, usecols=[0,1,2], nrows=wks.row_count, header=None, skiprows=1, evaluate_formulas=True)
            print('Spreadsheet '+row['SPREADSHEET_NAME']+' loaded back to DataFrame!')

            df_out.columns = ['RN', 'TITLE', 'TITLE_ENG']
            df_out.to_sql('otodom_data_flatten_translate', con=engine, if_exists='append', index=False, chunksize=16000, method=pd_writer)

    except Exception as e:
        print('--- Error --- ',e)
    finally:
        conn.close()
engine.dispose()

print("--- %s seconds ---" % (time.time() - start_time))